In [2]:
import os
import librosa
import numpy as np
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from shutil import copyfile
from PIL import Image
import matplotlib.pyplot as plt
import librosa.display

main_folder = '11/uploads2'
def create_spectrogram(audio_path, output_folder):
    y, sr = librosa.load(audio_path)
    D = librosa.amplitude_to_db(librosa.stft(y), ref=np.max)
    
    plt.figure(figsize=(10, 6))
    librosa.display.specshow(D, sr=sr, x_axis=None, y_axis=None)  
    plt.axis('off')  
    
    relative_path = os.path.relpath(audio_path, main_folder)
    subfolder = os.path.dirname(relative_path)
    
    output_path = os.path.join(output_folder, subfolder)
    os.makedirs(output_path, exist_ok=True)
    
    output_file = os.path.join(output_path, '{}.png'.format(os.path.splitext(os.path.basename(audio_path))[0]))
    plt.savefig(output_file, bbox_inches='tight', pad_inches=0)
    plt.close()

audio_dir = "11/uploads2"
output_directory = "spectrogram_images"
os.makedirs(output_directory, exist_ok=True)

for root, dirs, files in os.walk(main_folder):
    for file in files:
        if file.endswith(('.mp3', '.wav')): 
            audio_path = os.path.join(root, file)
            create_spectrogram(audio_path, output_folder=output_directory)

C:\Users\jospa\AppData\Local\Temp\ipykernel_171872\3797724238.py:15: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  D = librosa.amplitude_to_db(librosa.stft(y), ref=np.max)


In [15]:
from glob import glob
def load_spectrogram(filename, grayscale=True):
    image = Image.open(filename)
    if grayscale:
        image = image.convert("L")  # Convert to grayscale if needed
    image_array = np.array(image)
    image_vector = image_array.flatten()  # Reshape based on your image representation
    return image_vector


def cluster_spectrograms(image_paths, num_clusters, methods=["kmeans"]):
    features = []
    for image_path in image_paths:
        features.append(load_spectrogram(image_path))
    features_np = np.vstack(features)

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features_np)


    cluster_results = {}
    for method in methods:
        if method == "kmeans":
            clustering = KMeans(n_clusters=num_clusters)
        elif method == "spectral":
            clustering = SpectralClustering(n_clusters=num_clusters, eigen_solver="arpack")
        elif method == "hierarchical":
            clustering = AgglomerativeClustering(n_clusters=num_clusters, linkage="ward")
        else:
            raise ValueError(f"Invalid clustering method: {method}")

        clustering.fit(features_scaled)
        cluster_labels = clustering.labels_

        silhouette = silhouette_score(features_scaled, cluster_labels)

        cluster_results[method] = {"cluster_labels": cluster_labels, "silhouette_score": silhouette}

    return cluster_results


def print_image_cluster_mapping(image_paths, cluster_labels):

  if len(image_paths) != len(cluster_labels):
    raise ValueError("Length of image paths and cluster labels must be equal.")

  cluster_mappings = {}  # Dictionary to store image names for each cluster
  for i, image_path in enumerate(image_paths):
    cluster = cluster_labels[i]
    image_name = os.path.basename(image_path)
    if cluster not in cluster_mappings:
      cluster_mappings[cluster] = []
    cluster_mappings[cluster].append(image_name)

  # Print image names for each cluster
  for cluster, image_names in cluster_mappings.items():
    print(f"\nCluster {cluster}:")
    for name in image_names:
      print(f"  - {name}")



# Replace with your directory containing spectrogram images
spectrogram_dir = output_directory
image_paths = glob(os.path.join(spectrogram_dir, "*.png"))  # Assuming PNG format


# Choose desired number of clusters and methods to evaluate
num_clusters = 5
methods = ["kmeans", "spectral", "hierarchical"]  # Experiment with different combinations

# Perform clustering with evaluation
cluster_results = cluster_spectrograms(image_paths, num_clusters, methods)

# Print clustering results and silhouette scores
for method, result in cluster_results.items():
    print(f"Clustering Method: {method}")
    print(f"  Silhouette Score: {result['silhouette_score']}")

best_method = max(cluster_results.items(), key=lambda x: x[1]["silhouette_score"])[0]
print(cluster_results[best_method])



c:\Users\jospa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\jospa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


Clustering Method: kmeans
  Silhouette Score: 0.03808939965057224
Clustering Method: spectral
  Silhouette Score: -0.0822194641179427
Clustering Method: hierarchical
  Silhouette Score: 0.051403995940961476
{'cluster_labels': array([0, 3, 4, 1, 0, 1, 2, 0, 0, 0], dtype=int64), 'silhouette_score': 0.051403995940961476}


In [16]:
def print_image_cluster_mapping(image_paths, cluster_labels):

  if len(image_paths) != len(cluster_labels):
    raise ValueError("Length of image paths and cluster labels must be equal.")

  cluster_mappings = {}  # Dictionary to store image names for each cluster
  for i, image_path in enumerate(image_paths):
    cluster = cluster_labels[i]
    image_name = os.path.basename(image_path)
    if cluster not in cluster_mappings:
      cluster_mappings[cluster] = []
    cluster_mappings[cluster].append(image_name)

  # Print image names for each cluster
  for cluster, image_names in cluster_mappings.items():
    print(f"\nCluster {cluster}:")
    for name in image_names:
      print(f"  - {name}")
audio_files = [os.path.join(audio_dir, file) for file in os.listdir(audio_dir) if file.endswith(".mp3") or file.endswith(".wav")]
output_dir="output"
for audio_file, cluster_label in zip(audio_files, cluster_results[best_method]['cluster_labels']):
    destination_folder = os.path.join(output_dir, f"cluster_{cluster_label}")
    os.makedirs(destination_folder, exist_ok=True)
    filename = os.path.basename(audio_file)
    print(f"Copying {audio_file} to {destination_folder}")
    copyfile(audio_file, os.path.join(destination_folder, filename))

Copying ../music/uploads/new\Kerala-Song-MassTamilan.org.mp3 to output\cluster_0
Copying ../music/uploads/new\Life-turns-upside-down---Naan-Sirithal-Title-MassTamilan.io.mp3 to output\cluster_3
Copying ../music/uploads/new\LoveToday.mp3 to output\cluster_4
Copying ../music/uploads/new\Maryan.mp3 to output\cluster_1
Copying ../music/uploads/new\Mr.-Local-Theme-MassTamilan.org.mp3 to output\cluster_0
Copying ../music/uploads/new\Nadika-arambichiten-MassTamilan.io.mp3 to output\cluster_1
Copying ../music/uploads/new\Nelson-Manikkam-MassTamilan.io.mp3 to output\cluster_2
Copying ../music/uploads/new\Oxygen.mp3 to output\cluster_0
Copying ../music/uploads/new\Vaadi-Nee-Vaadi-MassTamilan.com.mp3 to output\cluster_0
Copying ../music/uploads/new\Yaara-Comali-MassTamilan.org.mp3 to output\cluster_0


In [17]:
print_image_cluster_mapping(image_paths, cluster_results[best_method]['cluster_labels'])


Cluster 0:
  - Kerala-Song-MassTamilan.org.png
  - Mr.-Local-Theme-MassTamilan.org.png
  - Oxygen.png
  - Vaadi-Nee-Vaadi-MassTamilan.com.png
  - Yaara-Comali-MassTamilan.org.png

Cluster 3:
  - Life-turns-upside-down---Naan-Sirithal-Title-MassTamilan.io.png

Cluster 4:
  - LoveToday.png

Cluster 1:
  - Maryan.png
  - Nadika-arambichiten-MassTamilan.io.png

Cluster 2:
  - Nelson-Manikkam-MassTamilan.io.png


In [12]:
from os import listdir
from os.path import isfile, join
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
model_path = 'new_classify.h5' 
class_labels = ['Anirudh', 'AR-Rahuman', 'GV', 'Hiphop-Tamizha', 'Vijay-Antony', 'Yuvan']
loaded_model = load_model(model_path)
def predict_song(spectrogram_path, file_name):
    img = image.load_img(spectrogram_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0

    predictions = loaded_model.predict(img_array)

    max_index = np.argmax(predictions[0])
    predicted_label = class_labels[max_index]
    probability = predictions[0][max_index]

    result = {
        'file_name': file_name,
        'predicted_class': predicted_label,
        'probability': float(probability)
    }
    return result

def predict_all_images_in_directory(directory):
    predictions = []

    files = [f for f in listdir(directory) if isfile(join(directory, f))]

    for file in files:
        image_path = join(directory, file)
        print(f"Predicting for image: {image_path}")
        prediction_result = predict_song(image_path, file)
        predictions.append(prediction_result)
    print(predictions)
    return predictions
directory="spectrogram_images"
predict_all_images_in_directory(directory)

Predicting for image: spectrogram_images\Kerala-Song-MassTamilan.org.png
1/1 [==============================] - 1s 877ms/step
Predicting for image: spectrogram_images\Life-turns-upside-down---Naan-Sirithal-Title-MassTamilan.io.png
1/1 [==============================] - 0s 78ms/step
Predicting for image: spectrogram_images\LoveToday.png
1/1 [==============================] - 0s 82ms/step
Predicting for image: spectrogram_images\Maryan.png
1/1 [==============================] - 0s 88ms/step
Predicting for image: spectrogram_images\Mr.-Local-Theme-MassTamilan.org.png
1/1 [==============================] - 0s 82ms/step
Predicting for image: spectrogram_images\Nadika-arambichiten-MassTamilan.io.png
1/1 [==============================] - 0s 80ms/step
Predicting for image: spectrogram_images\Nelson-Manikkam-MassTamilan.io.png
1/1 [==============================] - 0s 81ms/step
Predicting for image: spectrogram_images\Oxygen.png
1/1 [==============================] - 0s 79ms/step
Predicting fo

[{'file_name': 'Kerala-Song-MassTamilan.org.png',
  'predicted_class': 'Hiphop-Tamizha',
  'probability': 0.45823898911476135},
 {'file_name': 'Life-turns-upside-down---Naan-Sirithal-Title-MassTamilan.io.png',
  'predicted_class': 'Hiphop-Tamizha',
  'probability': 0.20129695534706116},
 {'file_name': 'LoveToday.png',
  'predicted_class': 'Vijay-Antony',
  'probability': 0.781818151473999},
 {'file_name': 'Maryan.png',
  'predicted_class': 'Vijay-Antony',
  'probability': 0.7410894632339478},
 {'file_name': 'Mr.-Local-Theme-MassTamilan.org.png',
  'predicted_class': 'Hiphop-Tamizha',
  'probability': 0.468331515789032},
 {'file_name': 'Nadika-arambichiten-MassTamilan.io.png',
  'predicted_class': 'Anirudh',
  'probability': 0.38702666759490967},
 {'file_name': 'Nelson-Manikkam-MassTamilan.io.png',
  'predicted_class': 'Hiphop-Tamizha',
  'probability': 0.6448734402656555},
 {'file_name': 'Oxygen.png',
  'predicted_class': 'Hiphop-Tamizha',
  'probability': 0.4629027545452118},
 {'file_